# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122060e+02     1.546567e+00
 * time: 0.08457803726196289
     1     1.079038e+01     1.008570e+00
 * time: 0.9184379577636719
     2    -1.256071e+01     1.015411e+00
 * time: 1.050055980682373
     3    -3.445638e+01     7.644100e-01
 * time: 1.2074689865112305
     4    -4.827367e+01     5.278907e-01
 * time: 1.3491511344909668
     5    -5.711799e+01     1.776993e-01
 * time: 1.4860920906066895
     6    -5.981573e+01     1.189209e-01
 * time: 1.5890710353851318
     7    -6.083877e+01     5.639945e-02
 * time: 1.6979451179504395
     8    -6.124802e+01     7.793251e-02
 * time: 1.8056659698486328
     9    -6.158612e+01     3.630137e-02
 * time: 1.9088599681854248
    10    -6.181275e+01     2.660409e-02
 * time: 2.017781972885132
    11    -6.197543e+01     2.166842e-02
 * time: 2.1246790885925293
    12    -6.204137e+01     1.932414e-02
 * time: 2.2251570224761963
    13    -6.210439e+01     1.811439e-02
 * time: 2.33320212364

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671065
    AtomicLocal         -18.8557701
    AtomicNonlocal      14.8522668
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485376 
    Xc                  -19.3336821

    total               -62.261666456593
